-----

https://en.cppreference.com/w/cpp/language/virtual

https://stackoverflow.com/questions/39932391/virtual-override-or-both-c

-----

### with or without keyword `virtual` [⬀](https://en.cppreference.com/w/cpp/language/virtual)

If some member function `vf` is declared as `virtual` in a class `Base`, and some class `Derived`, which is derived, directly or indirectly, from `Base`, has a declaration for member function with the same

* name
* parameter type list (but not the return type)
* cv-qualifiers
* ref-qualifiers

Then this function in the class `Derived` is also *`virtual`* (whether or not the keyword `virtual` is used in its declaration) and *overrides* `Base::vf` (whether or not the word `override` is used in its declaration).

`Base::vf` does not need to be visible (can be declared private, or inherited using private inheritance) to be overridden.

...

A function with the same name but different parameter list does not override the base function of the same name, but *hides* it: when [unqualified name lookup](https://en.cppreference.com/w/cpp/language/lookup) examines the scope of the derived class, the lookup finds the declaration and does not examine the base class.


### Virtual destructor [⬀](https://en.cppreference.com/w/cpp/language/virtual)

Even though destructors are not inherited, if a base class declares its destructor `virtual`, the derived destructor always overrides it. This makes it possible to delete dynamically allocated objects of polymorphic type through pointers to base.

Moreover, if a class is *polymorphic* (declares or inherits at least one virtual function), and its destructor is not virtual, deleting it is *undefined behavior* regardless of whether there are resources that would be leaked if the derived destructor is not invoked.

A useful guideline is that the destructor of any base class must be [public and virtual or protected and non-virtual](https://github.com/isocpp/CppCoreGuidelines/blob/master/CppCoreGuidelines.md#discussion-make-base-class-destructors-public-and-virtual-or-protected-and-nonvirtual).

-----

In [1]:
#include <iostream>
#include <typeinfo>

using namespace std;

In [2]:
class Animal
{
public:
            Animal(void)  { cout << "Animal born." << endl; }
    virtual ~Animal(void) { cout << "Animal dies." << endl; }

            void eat000(void) { cout << "Animal eats 000." << endl; }
            void eat001(void) { cout << "Animal eats 001." << endl; }
            void eat010(void) { cout << "Animal eats 010." << endl; }
            void eat011(void) { cout << "Animal eats 011." << endl; }
    virtual void eat100(void) { cout << "Animal eats 100.    (explicitly virtual)" << endl; }
    virtual void eat101(void) { cout << "Animal eats 101.    (explicitly virtual)" << endl; }
    virtual void eat110(void) { cout << "Animal eats 110.    (explicitly virtual)" << endl; }
    virtual void eat111(void) { cout << "Animal eats 111.    (explicitly virtual)" << endl; }
};

In [3]:
class Dog : public Animal
{
public:
    Dog(void)  { cout << "\tDog born." << endl; }
    ~Dog(void) { cout << "\tDog dies. (automatically virtual)" << endl; }

            void eat000(void)          { cout << "\tDog eats 000." << endl; }
            void eat001(void)          { cout << "\tDog eats 001." << endl; }
    virtual void eat010(void)          { cout << "\tDog eats 010.    (explicitly virtual)" << endl; }
    virtual void eat011(void)          { cout << "\tDog eats 011.    (explicitly virtual)" << endl; }
            void eat100(void) override { cout << "\tDog eats 100. (automatically virtual)" << endl; }
            void eat101(void) override { cout << "\tDog eats 101. (automatically virtual)" << endl; }
    virtual void eat110(void) override { cout << "\tDog eats 110.    (explicitly virtual)" << endl; }
    virtual void eat111(void) override { cout << "\tDog eats 111.    (explicitly virtual)" << endl; }
};

☝︎ note: `eat100` and `eat101` are virtual too

In [4]:
class Husky : public Dog
{
public:
    Husky(void)  { cout << "\t\tHusky born.\n"; }
    ~Husky(void) { cout << "\t\tHusky dies. (automatically virtual)\n"; }

            void eat000(void)          { cout << "\t\tHusky eats 000." << endl; }
    virtual void eat001(void)          { cout << "\t\tHusky eats 001.    (explicitly virtual)" << endl; }
            void eat010(void) override { cout << "\t\tHusky eats 010. (automatically virtual)" << endl; }
    virtual void eat011(void) override { cout << "\t\tHusky eats 011.    (explicitly virtual)" << endl; }
            void eat100(void) override { cout << "\t\tHusky eats 100. (automatically virtual)" << endl; }
    virtual void eat101(void) override { cout << "\t\tHusky eats 101.    (explicitly virtual)" << endl; }
            void eat110(void) override { cout << "\t\tHusky eats 110. (automatically virtual)" << endl; }
    virtual void eat111(void) override { cout << "\t\tHusky eats 111.    (explicitly virtual)" << endl; }
};

-----

In [5]:
template <class T>
void done_by(T * p)
{
    cout << "------------------------------ <" << typeid(T).name() << ">\n";

    p->eat000();
    p->eat001();
    p->eat010();
    p->eat011();
    p->eat100();
    p->eat101();
    p->eat110();
    p->eat111();

    cout << endl << "explicitly an animal" << endl;

    p->Animal::eat000();
    p->Animal::eat001();
    p->Animal::eat010();
    p->Animal::eat011();
    p->Animal::eat100();
    p->Animal::eat101();
    p->Animal::eat110();
    p->Animal::eat111();

    cout << endl;
}

-----

# `new Husky`

## as `Husky`

In [6]:
Husky * husky_husky = new Husky;
done_by(husky_husky);
delete husky_husky;

Animal born.
	Dog born.
		Husky born.
------------------------------ <5Husky>
		Husky eats 000.
		Husky eats 001.    (explicitly virtual)
		Husky eats 010. (automatically virtual)
		Husky eats 011.    (explicitly virtual)
		Husky eats 100. (automatically virtual)
		Husky eats 101.    (explicitly virtual)
		Husky eats 110. (automatically virtual)
		Husky eats 111.    (explicitly virtual)

explicitly an animal
Animal eats 000.
Animal eats 001.
Animal eats 010.
Animal eats 011.
Animal eats 100.    (explicitly virtual)
Animal eats 101.    (explicitly virtual)
Animal eats 110.    (explicitly virtual)
Animal eats 111.    (explicitly virtual)

		Husky dies. (automatically virtual)
	Dog dies. (automatically virtual)
Animal dies.


## as `Dog`

In [7]:
Dog * dog_husky = new Husky;
done_by(dog_husky);
delete dog_husky;

Animal born.
	Dog born.
		Husky born.
------------------------------ <3Dog>
	Dog eats 000.
	Dog eats 001.
		Husky eats 010. (automatically virtual)
		Husky eats 011.    (explicitly virtual)
		Husky eats 100. (automatically virtual)
		Husky eats 101.    (explicitly virtual)
		Husky eats 110. (automatically virtual)
		Husky eats 111.    (explicitly virtual)

explicitly an animal
Animal eats 000.
Animal eats 001.
Animal eats 010.
Animal eats 011.
Animal eats 100.    (explicitly virtual)
Animal eats 101.    (explicitly virtual)
Animal eats 110.    (explicitly virtual)
Animal eats 111.    (explicitly virtual)

		Husky dies. (automatically virtual)
	Dog dies. (automatically virtual)
Animal dies.


## as `Animal`

In [8]:
Animal * animal_husky = new Husky;
done_by(animal_husky);
delete animal_husky;

Animal born.
	Dog born.
		Husky born.
------------------------------ <6Animal>
Animal eats 000.
Animal eats 001.
Animal eats 010.
Animal eats 011.
		Husky eats 100. (automatically virtual)
		Husky eats 101.    (explicitly virtual)
		Husky eats 110. (automatically virtual)
		Husky eats 111.    (explicitly virtual)

explicitly an animal
Animal eats 000.
Animal eats 001.
Animal eats 010.
Animal eats 011.
Animal eats 100.    (explicitly virtual)
Animal eats 101.    (explicitly virtual)
Animal eats 110.    (explicitly virtual)
Animal eats 111.    (explicitly virtual)

		Husky dies. (automatically virtual)
	Dog dies. (automatically virtual)
Animal dies.


## as `Dog` as `Animal`

In [9]:
Dog * dog_husky2 = new Husky;
Animal * animal_dog_husky = dog_husky2;
done_by(animal_dog_husky);
delete animal_dog_husky;

Animal born.
	Dog born.
		Husky born.
------------------------------ <6Animal>
Animal eats 000.
Animal eats 001.
Animal eats 010.
Animal eats 011.
		Husky eats 100. (automatically virtual)
		Husky eats 101.    (explicitly virtual)
		Husky eats 110. (automatically virtual)
		Husky eats 111.    (explicitly virtual)

explicitly an animal
Animal eats 000.
Animal eats 001.
Animal eats 010.
Animal eats 011.
Animal eats 100.    (explicitly virtual)
Animal eats 101.    (explicitly virtual)
Animal eats 110.    (explicitly virtual)
Animal eats 111.    (explicitly virtual)

		Husky dies. (automatically virtual)
	Dog dies. (automatically virtual)
Animal dies.


# `new Dog`

## as `Dog`

In [10]:
Dog * dog_dog = new Dog;
done_by(dog_dog);
delete dog_dog;

Animal born.
	Dog born.
------------------------------ <3Dog>
	Dog eats 000.
	Dog eats 001.
	Dog eats 010.    (explicitly virtual)
	Dog eats 011.    (explicitly virtual)
	Dog eats 100. (automatically virtual)
	Dog eats 101. (automatically virtual)
	Dog eats 110.    (explicitly virtual)
	Dog eats 111.    (explicitly virtual)

explicitly an animal
Animal eats 000.
Animal eats 001.
Animal eats 010.
Animal eats 011.
Animal eats 100.    (explicitly virtual)
Animal eats 101.    (explicitly virtual)
Animal eats 110.    (explicitly virtual)
Animal eats 111.    (explicitly virtual)

	Dog dies. (automatically virtual)
Animal dies.


## as `Animal`

In [11]:
Animal * animal_dog = new Dog;
done_by(animal_dog);
delete animal_dog;

Animal born.
	Dog born.
------------------------------ <6Animal>
Animal eats 000.
Animal eats 001.
Animal eats 010.
Animal eats 011.
	Dog eats 100. (automatically virtual)
	Dog eats 101. (automatically virtual)
	Dog eats 110.    (explicitly virtual)
	Dog eats 111.    (explicitly virtual)

explicitly an animal
Animal eats 000.
Animal eats 001.
Animal eats 010.
Animal eats 011.
Animal eats 100.    (explicitly virtual)
Animal eats 101.    (explicitly virtual)
Animal eats 110.    (explicitly virtual)
Animal eats 111.    (explicitly virtual)

	Dog dies. (automatically virtual)
Animal dies.


-----

# What won't work

## only virtual member functions can be marked '`override`', so this won't work: 

In [12]:
class BadDog : public Animal
{
public:
    BadDog(void)  { cout << "\tDog born." << endl; }
    ~BadDog(void) { cout << "\tDog dies." << endl; }

            void eat000(void) override { cout << "\tDog eats 000." << endl; }
            void eat001(void) override { cout << "\tDog eats 001." << endl; }
    virtual void eat010(void) override { cout << "\tDog eats 010." << endl; }
    virtual void eat011(void) override { cout << "\tDog eats 011." << endl; }
            void eat100(void) override { cout << "\tDog eats 100." << endl; }
            void eat101(void) override { cout << "\tDog eats 101." << endl; }
    virtual void eat110(void) override { cout << "\tDog eats 110." << endl; }
    virtual void eat111(void) override { cout << "\tDog eats 111." << endl; }
};

input_line_19:6:31: error: only virtual member functions can be marked 'override'
            void eat000(void) override { cout << "\tDog eats 000." << endl; }
                              ^~~~~~~~~
input_line_19:7:31: error: only virtual member functions can be marked 'override'
            void eat001(void) override { cout << "\tDog eats 001." << endl; }
                              ^~~~~~~~~
input_line_19:8:18: error: 'eat010' marked 'override' but does not override any member functions
    virtual void eat010(void) override { cout << "\tDog eats 010." << endl; }
                 ^
input_line_19:9:18: error: 'eat011' marked 'override' but does not override any member functions
    virtual void eat011(void) override { cout << "\tDog eats 011." << endl; }
                 ^


Interpreter Error: 

-----